In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
spg.__version__

'1.2.7'

# 1. Read in data

In [5]:

#Read original data and save it to h5ad
from scanpy import read_10x_h5
adata = read_10x_h5("../tutorial/data/151510/expression_matrix.h5")
spatial=pd.read_csv("../tutorial/data/151510/positions.txt",sep=",",header=None,na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]
adata.obs["x2"]=spatial[2]
adata.obs["x3"]=spatial[3]
adata.obs["x4"]=spatial[4]
adata.obs["x5"]=spatial[5]
adata.obs["x_array"]=adata.obs["x2"]
adata.obs["y_array"]=adata.obs["x3"]
adata.obs["x_pixel"]=adata.obs["x4"]
adata.obs["y_pixel"]=adata.obs["x5"]

#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad("../tutorial/data/151510/sample_data.h5ad")

#Read in gene expression and spatial location
adata=sc.read("../tutorial/data/151510/sample_data.h5ad")
#Read in hitology image
img=cv2.imread("../tutorial/data/151510/histology.png")

# 2. Integrate gene expression and histology into a Graph

In [6]:
#Set coordinates
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()

#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img_new[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('./sample_results/151510_map.jpg', img_new)

True

In [7]:
#Calculate adjacent matrix
s=1
b=49
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
#adj=calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)

# 定义文件路径
file_path = './data/151510/adj.csv'

# 检查目录是否存在，如果不存在则创建
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 保存文件
np.savetxt(file_path, adj, delimiter=',')

print("邻接矩阵已保存到:", file_path)

Calculateing adj matrix using histology image...
Var of c0,c1,c2 =  nan nan nan
Var of x,y,z =  6511733.839876174 6135285.956270563 nan
邻接矩阵已保存到: ./data/151510/adj.csv


# 3. Spatial domain detection using SpaGCN

In [ ]:
adata=sc.read("../tutorial/data/151510/sample_data.h5ad")
adj=np.loadtxt('./data/151510/adj.csv', delimiter=',')
adata.var_names_make_unique()
spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
spg.prefilter_specialgenes(adata)
#Normalize and take log for UMI
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

### Set hyper-parameters

In [ ]:
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)

In [4]:
#If the number of clusters known, we can use the spg.search_res() fnction to search for suitable resolution(optional)
#For this toy data, we set the number of clusters=7 since this tissue has 7 layers
n_clusters=7
#Set seed
r_seed=t_seed=n_seed=100
#Seaech for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

NameError: name 'adata' is not defined

### Run SpaGCN

In [ ]:
# Initialize SpaGCN classifier
clf = spg.SpaGCN()
clf.set_l(l)

# Set seeds for reproducibility
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
print("Seeds set: random={}, torch={}, numpy={}".format(r_seed, t_seed, n_seed))

# Train the model
print("Starting training...")
clf.train(adata, adj, init_spa=True, init="louvain", res=res, tol=5e-3, lr=0.05, max_epochs=200)
print("Training completed.")

# Predict clusters
y_pred, prob = clf.predict()
print("Prediction completed. Unique predictions:", np.unique(y_pred))

# Store predictions in adata
adata.obs["pred"] = y_pred
adata.obs["pred"] = adata.obs["pred"].astype('category')
print("Predictions stored in adata.obs['pred'].")

# Optional: Cluster refinement
print("Starting cluster refinement...")
adj_2d = spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
print("2D adjacency matrix calculated.")

refined_pred = spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
print("Cluster refinement completed. Unique refined predictions:", np.unique(refined_pred))

# Store refined predictions in adata
adata.obs["refined_pred"] = refined_pred
adata.obs["refined_pred"] = adata.obs["refined_pred"].astype('category')
print("Refined predictions stored in adata.obs['refined_pred'].")

# Save results
output_path = "./151510/sample_results/results.h5ad"
print("Saving results to:", output_path)
adata.write_h5ad(output_path)
print("Results saved successfully.")

In [ ]:
adata = sc.read_h5ad("./151510/sample_results/results.h5ad")
print(adata.obs.columns.tolist())

In [ ]:
adata.obsm["spatial"] = adata.obs[["x_array", "y_array"]].values

In [ ]:
from spatial_metrics import compute_NMI, compute_HOM, compute_COM, compute_ASW, compute_CHAOS, compute_PAS

refined_pred_key= "refined_pred"

print("ASW:   ", compute_ASW(adata, refined_pred_key))
print("CHAOS: ", compute_CHAOS(adata, refined_pred_key))
print("PAS:   ", compute_PAS(adata, refined_pred_key))

In [ ]:
# ─── 11.1 评估：加载 ground‑truth → 对齐 → 计算 ARI／NMI ─────────────────────────

from sklearn import metrics
import pandas as pd

# 1）设置路径与数据集标识
file_fold = "../tutorial/data/151510"      # 你的 layer 注释 CSV 所在文件夹
dataset   = "151510"

print("Loading ground truth annotations...")
# 2）读取层注释 CSV（需包含“barcode”和“layer”两列）
df_meta = pd.read_csv(f"{file_fold}/{dataset}_layer_annotations.csv")
print(f"Layer annotations shape: {df_meta.shape}")
print(f"AnnData shape: {adata.shape}")

# 3）对齐 AnnData 和元数据
meta_index  = df_meta['barcode'].tolist()
adata_index = adata.obs.index.tolist()
common_spots = set(meta_index) & set(adata_index)
print(f"Common spots between layer annotations and data: {len(common_spots)}")

# 4）如果索引不一致，先筛一下 AnnData
if len(meta_index) != len(adata_index):
    print("Filtering AnnData to match layer annotations...")
    adata = adata[adata.obs.index.isin(meta_index)].copy()
    print(f"After filtering, AnnData shape: {adata.shape}")

# 5）严格按照 barcodes 重新排序元数据
df_meta = df_meta.set_index('barcode').loc[adata.obs.index]

# 6）把真值写入 obs
adata.obs['ground_truth'] = df_meta['layer'].values

# 7）去除 NA
adata = adata[~adata.obs['ground_truth'].isna()].copy()
print(f"After dropping NA, AnnData shape: {adata.shape}")

# 8）计算指标（这里把 'pred' 作为 domain 列，如果是其它名字请改成对应列）
ARI = metrics.adjusted_rand_score(
    adata.obs['pred'], adata.obs['ground_truth']
)
NMI = metrics.normalized_mutual_info_score(
    adata.obs['pred'], adata.obs['ground_truth']
)
adata.uns['ARI'] = ARI
adata.uns['NMI'] = NMI

print('Dataset:', dataset)
print('ARI:    ', ARI)
print('NMI:    ', NMI)

===============分割线===============

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

# Read the data
adata_path = "../tutorial/sample_results/results.h5ad"
print(f"Reading data from: {adata_path}")
adata = sc.read(adata_path)
print("Data read successfully.")
print(f"Number of cells: {adata.shape[0]}, Number of genes: {adata.shape[1]}")

# Set colors used
plot_color = ["#F56867", "#FEB915", "#C798EE", "#59BE86", "#7495D3", "#D1D1D1", "#6D1A9C", "#15821E", "#3A84E6", "#997273", "#787878", "#DB4C6C", "#9E7A7A", "#554236", "#AF5F3C", "#93796C", "#F9BD3F", "#DAB370", "#877F6C", "#268785"]
print("Color palette set.")

# Plot spatial domains
domains = "pred"
num_celltype = len(adata.obs[domains].unique())
print(f"Number of unique cell types in '{domains}': {num_celltype}")
adata.uns[domains + "_colors"] = list(plot_color[:num_celltype])
print(f"Colors assigned to '{domains}'.")

print(f"Plotting spatial domains for '{domains}'...")
ax = sc.pl.scatter(adata, alpha=1, x="y_pixel", y="x_pixel", color=domains, title=domains, color_map=plot_color, show=False, size=100000 / adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
output_path = "./sample_results/pred.png"
print(f"Saving plot to: {output_path}")
plt.savefig(output_path, dpi=600)
plt.close()
print(f"Plot saved successfully for '{domains}'.")

# Plot refined spatial domains
domains = "refined_pred"
num_celltype = len(adata.obs[domains].unique())
print(f"Number of unique cell types in '{domains}': {num_celltype}")
adata.uns[domains + "_colors"] = list(plot_color[:num_celltype])
print(f"Colors assigned to '{domains}'.")

print(f"Plotting refined spatial domains for '{domains}'...")
ax = sc.pl.scatter(adata, alpha=1, x="y_pixel", y="x_pixel", color=domains, title=domains, color_map=plot_color, show=False, size=100000 / adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
output_path = "./sample_results/refined_pred.png"
print(f"Saving plot to: {output_path}")
plt.savefig(output_path, dpi=600)
plt.close()
print(f"Plot saved successfully for '{domains}'.")

### Identify SVGs

In [ ]:
#Read in raw data
# raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x2"]
raw.obs["y_array"]=raw.obs["x3"]
raw.obs["x_pixel"]=raw.obs["x4"]
raw.obs["y_pixel"]=raw.obs["x5"]
#Convert sparse matrix to non-sparse
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)

In [ ]:
#Use domain 0 as an example
target=0
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=raw.obs.index.tolist(), 
                                   x=raw.obs["x_array"].tolist(), 
                                   y=raw.obs["y_array"].tolist(), 
                                   pred=raw.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)

nbr_domians=nbr_domians[0:3]
de_genes_info=spg.rank_genes_groups(input_adata=raw,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info=de_genes_info
filtered_info=filtered_info[(filtered_info["pvals_adj"]<0.05) &
                            (filtered_info["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info["fold_change"]>min_fold_change)]
filtered_info=filtered_info.sort_values(by="in_group_fraction", ascending=False)
filtered_info["target_dmain"]=target
filtered_info["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info["genes"].tolist())

In [ ]:
filtered_info

In [ ]:
#Plot refinedspatial domains
color_self = clr.LinearSegmentedColormap.from_list('pink_green', ['#3AB370',"#EAE7CC","#FD1593"], N=256)
for g in filtered_info["genes"].tolist():
    raw.obs["exp"]=raw.X[:,raw.var.index==g]
    ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=g,color_map=color_self,show=False,size=100000/raw.shape[0])
    ax.set_aspect('equal', 'box')
    ax.axes.invert_yaxis()
    plt.savefig("./sample_results/"+g+".png", dpi=600)
    plt.close()

### Identify Meta Gene

In [ ]:
#Use domain 2 as an example
target=2
meta_name, meta_exp=spg.find_meta_gene(input_adata=raw,
                    pred=raw.obs["pred"].tolist(),
                    target_domain=target,
                    start_gene="GFAP",
                    mean_diff=0,
                    early_stop=True,
                    max_iter=3,
                    use_raw=False)

raw.obs["meta"]=meta_exp

In [ ]:
#Plot meta gene
g="GFAP"
raw.obs["exp"]=raw.X[:,raw.var.index==g]
ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=g,color_map=color_self,show=False,size=100000/raw.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/"+g+".png", dpi=600)
plt.close()

raw.obs["exp"]=raw.obs["meta"]
ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=meta_name,color_map=color_self,show=False,size=100000/raw.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/meta_gene.png", dpi=600)
plt.close()

### Multiple tissue sections analysis

In [ ]:
adata1=sc.read("./tutorial/data/Mouse_brain/MA1.h5ad")
adata2=sc.read("../tutorial/data/Mouse_brain/MP1.h5ad")
img1=cv2.imread("../tutorial/data/Mouse_brain/MA1_histology.tif")
img2=cv2.imread("../tutorial/data/Mouse_brain/MP1_histology.tif")